In [2]:
# Library Import

import os
import sys
%pwd

'f:\\My_Design\\Data_Science\\Data Science\\Data Science Notes\\Machine Learning\\Projects\\End-to-End Projects\\End-to-End-Machine-Learning-Project-with-Deployment\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'f:\\My_Design\\Data_Science\\Data Science\\Data Science Notes\\Machine Learning\\Projects\\End-to-End Projects\\End-to-End-Machine-Learning-Project-with-Deployment'

In [ ]:
import dagshub
dagshub.init(repo_owner='kunalkalkotwar21', repo_name='End-to-End-Machine-Learning-Project-with-Deployment', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

Output()

In [5]:
os.environ['MLFLOW_TRACKING_URI']='https://dagshub.com/kunalkalkotwar21/End-to-End-Machine-Learning-Project-with-Deployment.mlflow'
os.environ['MLFLOW_TRACKING_USERNAME']='kunalkalkotwar21'
os.environ['MLFLOW_TRACKING_PASSWORD']='e493d450b8e9059d8c01a75ebb5b404bb27528b1'

In [6]:
# Perparing the model evaluation entities [config_entity]

from dataclasses import dataclass
from pathlib import Path

# Below is the data class for data Validation configuration
# This class is used to define the configuration for data Validation in a machine learning pipeline.
@dataclass(frozen=True)
class ModelEvaluationConfig:
    """Model Evaluation Configuration"""
    # Define the attributes of the class
    root_dir: Path
    model_path: Path
    test_data_path: Path
    report_path: Path
    all_param: dict
    target_column: str
    mlflow_tracking_uri: str

In [7]:
from ml_project.constants import *
from ml_project.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    """Configuration Manager"""
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH, 
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        """Initialize Configuration Manager"""
        # Initialize the ConfigurationManager with file paths for configuration, parameters, and schema.
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)



        # Create directories for artifacts and data ingestion
        create_directories([self.config.artifacts_root])  # Create the artifacts root directory

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        """Get Model Evaluation Configuration"""
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        
        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            model_path = config.model_path,
            test_data_path = config.test_data_path,
            report_path = config.report_path,
            all_param = params,
            target_column = schema,
            mlflow_tracking_uri = 'https://dagshub.com/kunalkalkotwar21/End-to-End-Machine-Learning-Project-with-Deployment.mlflow'
            )
        
        return model_evaluation_config

In [9]:
import os
import numpy as np
import pandas
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import joblib

In [ ]:
class ModelEvaluation:
    """Model Evaluation"""
    def __init__(self, config: ModelEvaluationConfig):
        """Initialize Model Evaluation"""
        self.config = config

    def eval_report(self, actual, predict):
        """Evaluation Report"""
        r2 = r2_score(actual, predict)
        rmse = np.sqrt(mean_squared_error(actual, predict))
        mae = mean_absolute_error(actual, predict)

        return r2, rmse, mae